## Question_9_Solution:

**Q-9.** A cyber security agent wants to check the Microsoft Malware so need he came
to you as a Machine learning Engineering with Data, You need to find the Malwareusing a supervised algorithm and you need to find the accuracy of the model.
Dataset This is the Dataset You can use this dataset for this question.

In [1]:
# import required libraries:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Provide the path to your CSV file in Google Drive
csv_path = '/content/drive/MyDrive/Colab Notebooks/Q9.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)
# Display the DataFrame
df.head(2)


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,00101159d08be0301ed6193eb126df84,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1609.0,0,7.0,0,NaN,53447.0,...,12416.0,1,NaN,0.0,1,0,0.0,0.0,11.0,1
1,007ae822f99412d25ccdddc48cb51c6b,win8defender,1.1.15200.1,4.10.209.0,1.275.1397.0,0,7.0,0,NaN,53447.0,...,52369.0,1,0.0,0.0,0,0,0.0,0.0,10.0,0


In [4]:
df.shape

(15000, 83)

In [5]:
df.columns

Index(['MachineIdentifier', 'ProductName', 'EngineVersion', 'AppVersion',
       'AvSigVersion', 'IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode',
       'DefaultBrowsersIdentifier', 'AVProductStatesIdentifier',
       'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm',
       'CountryIdentifier', 'CityIdentifier', 'OrganizationIdentifier',
       'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'Platform',
       'Processor', 'OsVer', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease',
       'OsBuildLab', 'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'PuaMode',
       'SMode', 'IeVerIdentifier', 'SmartScreen', 'Firewall', 'UacLuaenable',
       'Census_MDC2FormFactor', 'Census_DeviceFamily',
       'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier',
       'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_ProcessorClass',
       'Census_PrimaryDiskTotalCapacity', 'Census_PrimaryDiskTypeName',
       'Census_

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 83 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   MachineIdentifier                                  15000 non-null  object 
 1   ProductName                                        15000 non-null  object 
 2   EngineVersion                                      15000 non-null  object 
 3   AppVersion                                         15000 non-null  object 
 4   AvSigVersion                                       15000 non-null  object 
 5   IsBeta                                             15000 non-null  int64  
 6   RtpStateBitfield                                   14943 non-null  float64
 7   IsSxsPassiveMode                                   15000 non-null  int64  
 8   DefaultBrowsersIdentifier                          738 non-null    float64
 9   AVProd

In [7]:
column_dtypes = {'IsBeta' : 'Int8',
                 'RtpStateBitfield' : 'Int16',
                 'IsSxsPassiveMode' : 'Int8',
                 'HasTpm' : 'Int8',
                 'CountryIdentifier' : 'Int64',
                 'CityIdentifier' : 'Int64',
                 'OrganizationIdentifier' : 'Int64',
                 'IsProtected' : 'Int8',
                 'AutoSampleOptIn' : 'Int8',
                 'SMode' : 'Int8',
                 'Firewall' : 'Int8',
                 'Census_HasOpticalDiskDrive' : 'Int8',
                 'Census_IsPortableOperatingSystem' : 'Int8',
                 'Census_IsFlightsDisabled' : 'Int8',
                 'Census_IsSecureBootEnabled' : 'Int8',
                 'Census_IsWIMBootEnabled' : 'Int8',
                 'Census_IsVirtualDevice' : 'Int8',
                 'Census_IsTouchEnabled' : 'Int8',
                 'Census_IsPenCapable' : 'Int8',
                 'Census_IsAlwaysOnAlwaysConnectedCapable': 'Int8',
                 'Wdft_IsGamer' : 'Int8',
                 'HasDetections' : 'int'}

In [8]:
columns_to_ignore = ('ProductName',
                     'DefaultBrowsersIdentifier', 
                     'PuaMode',
                     'Census_ProcessorClass', 
                     'Census_InternalBatteryType', 
                     'Census_IsFlightingInternal', 
                     'Census_ThresholdOptIn', # 
                     
                     # numerical features
                     'Census_PrimaryDiskTotalCapacity', 
                     'Census_SystemVolumeTotalCapacity', 
                     'Census_TotalPhysicalRAM',        
                     'Census_InternalPrimaryDisplayResolutionHorizontal',
                     'Census_InternalPrimaryDisplayResolutionVertical',
                     'Census_InternalPrimaryDiagonalDisplaySizeInInches',
                     'Census_InternalBatteryNumberOfCharges',
                     'IsBeta', 
                     'AutoSampleOptIn', 
                     'UacLuaenable', 
                     'Census_IsWIMBootEnabled',
                     
                     'Census_FlightRing_not',
                     'Census_IsAlwaysOnAlwaysConnectedCapable',
                     'Census_IsSecureBootEnabled',
                     'Census_IsTouchEnabled',
                     'Census_IsVirtualDevice',
                     'SMode'
                    )

In [9]:
# label data:
label_col = 'HasDetections'

In [10]:
# Ignoring non-required columns:
df = pd.read_csv(csv_path, usecols = lambda x: x not in columns_to_ignore, dtype = column_dtypes)
df.head()

,MachineIdentifier,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,...,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightsDisabled,Census_FlightRing,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsPenCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,00101159d08be0301ed6193eb126df84,1.1.15200.1,4.18.1807.18075,1.275.1609.0,7,0,53447.0,1.0,1.0,1,...,IS_GENUINE,Retail,0,Retail,142.0,12416.0,0,0,11.0,1
1,007ae822f99412d25ccdddc48cb51c6b,1.1.15200.1,4.10.209.0,1.275.1397.0,7,0,53447.0,1.0,1.0,1,...,IS_GENUINE,Retail,0,Retail,807.0,52369.0,0,0,10.0,0
2,008bf0fe078bbaf98531e39e1940ae03,1.1.15200.1,4.18.1807.18075,1.275.1062.0,7,0,53447.0,1.0,1.0,1,...,IS_GENUINE,Retail,0,Retail,556.0,56932.0,0,0,1.0,0
3,001eebd2639759cc82b7921c0c36f707,1.1.15100.1,4.18.1806.18062,1.273.283.0,7,0,53447.0,1.0,1.0,1,...,IS_GENUINE,Retail,0,Retail,142.0,9888.0,0,1,7.0,0
4,00bec230bf2577fcda2d25b9223865b3,1.1.15200.1,4.10.209.0,1.275.314.0,7,0,279.0,4.0,3.0,0,...,IS_GENUINE,Retail,<NA>,Retail,142.0,13873.0,0,0,13.0,1


In [11]:
# Checking Unique and NA Values:
for col in df.columns:
    print(f'"{col}" has {df[col].nunique()} unique values and {df[col].isna().sum() / len(df) * 100:.5f}% NA values.')
    

"MachineIdentifier" has 15000 unique values and 0.00000% NA values.
"EngineVersion" has 35 unique values and 0.00000% NA values.
"AppVersion" has 59 unique values and 0.00000% NA values.
"AvSigVersion" has 1908 unique values and 0.00000% NA values.
"RtpStateBitfield" has 6 unique values and 0.38000% NA values.
"IsSxsPassiveMode" has 2 unique values and 0.00000% NA values.
"AVProductStatesIdentifier" has 651 unique values and 0.39333% NA values.
"AVProductsInstalled" has 5 unique values and 0.39333% NA values.
"AVProductsEnabled" has 4 unique values and 0.39333% NA values.
"HasTpm" has 2 unique values and 0.00000% NA values.
"CountryIdentifier" has 197 unique values and 0.00000% NA values.
"CityIdentifier" has 4752 unique values and 3.85333% NA values.
"OrganizationIdentifier" has 26 unique values and 30.32000% NA values.
"GeoNameIdentifier" has 193 unique values and 0.00000% NA values.
"LocaleEnglishNameIdentifier" has 140 unique values and 0.00000% NA values.
"Platform" has 4 unique v

In [12]:
df.isnull().sum()

MachineIdentifier                            0
EngineVersion                                0
AppVersion                                   0
AvSigVersion                                 0
RtpStateBitfield                            57
IsSxsPassiveMode                             0
AVProductStatesIdentifier                   59
AVProductsInstalled                         59
AVProductsEnabled                           59
HasTpm                                       0
CountryIdentifier                            0
CityIdentifier                             578
OrganizationIdentifier                    4548
GeoNameIdentifier                            0
LocaleEnglishNameIdentifier                  0
Platform                                     0
Processor                                    0
OsVer                                        0
OsBuild                                      0
OsSuite                                      0
OsPlatformSubRelease                         0
OsBuildLab   

In [13]:
df.nunique()

MachineIdentifier                         15000
EngineVersion                                35
AppVersion                                   59
AvSigVersion                               1908
RtpStateBitfield                              6
IsSxsPassiveMode                              2
AVProductStatesIdentifier                   651
AVProductsInstalled                           5
AVProductsEnabled                             4
HasTpm                                        2
CountryIdentifier                           197
CityIdentifier                             4752
OrganizationIdentifier                       26
GeoNameIdentifier                           193
LocaleEnglishNameIdentifier                 140
Platform                                      4
Processor                                     3
OsVer                                         4
OsBuild                                      26
OsSuite                                       4
OsPlatformSubRelease                    

In [14]:
df.columns

Index(['MachineIdentifier', 'EngineVersion', 'AppVersion', 'AvSigVersion',
       'RtpStateBitfield', 'IsSxsPassiveMode', 'AVProductStatesIdentifier',
       'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm',
       'CountryIdentifier', 'CityIdentifier', 'OrganizationIdentifier',
       'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'Platform',
       'Processor', 'OsVer', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease',
       'OsBuildLab', 'SkuEdition', 'IsProtected', 'IeVerIdentifier',
       'SmartScreen', 'Firewall', 'Census_MDC2FormFactor',
       'Census_DeviceFamily', 'Census_OEMNameIdentifier',
       'Census_OEMModelIdentifier', 'Census_ProcessorCoreCount',
       'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTypeName',
       'Census_HasOpticalDiskDrive', 'Census_ChassisTypeName',
       'Census_PowerPlatformRoleName', 'Census_OSVersion',
       'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSBuildNumber',
 

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 60 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   MachineIdentifier                       15000 non-null  object 
 1   EngineVersion                           15000 non-null  object 
 2   AppVersion                              15000 non-null  object 
 3   AvSigVersion                            15000 non-null  object 
 4   RtpStateBitfield                        14943 non-null  Int16  
 5   IsSxsPassiveMode                        15000 non-null  Int8   
 6   AVProductStatesIdentifier               14941 non-null  float64
 7   AVProductsInstalled                     14941 non-null  float64
 8   AVProductsEnabled                       14941 non-null  float64
 9   HasTpm                                  15000 non-null  Int8   
 10  CountryIdentifier                       15000 non-null  In

In [16]:
# Split features and Target:
X = df.drop(['MachineIdentifier','HasDetections'], axis = 1)
y = df['HasDetections']

In [17]:
X.columns

Index(['EngineVersion', 'AppVersion', 'AvSigVersion', 'RtpStateBitfield',
       'IsSxsPassiveMode', 'AVProductStatesIdentifier', 'AVProductsInstalled',
       'AVProductsEnabled', 'HasTpm', 'CountryIdentifier', 'CityIdentifier',
       'OrganizationIdentifier', 'GeoNameIdentifier',
       'LocaleEnglishNameIdentifier', 'Platform', 'Processor', 'OsVer',
       'OsBuild', 'OsSuite', 'OsPlatformSubRelease', 'OsBuildLab',
       'SkuEdition', 'IsProtected', 'IeVerIdentifier', 'SmartScreen',
       'Firewall', 'Census_MDC2FormFactor', 'Census_DeviceFamily',
       'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier',
       'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTypeName',
       'Census_HasOpticalDiskDrive', 'Census_ChassisTypeName',
       'Census_PowerPlatformRoleName', 'Census_OSVersion',
       'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSBuildNumber',
       'Census_OSBuildRevisio

In [18]:
# Preprocessing function:
import sys

def preprocess_cat_values(df):
    temp = df.copy()

    # Drop the 'OsBuildLab' column if it exists
    if 'OsBuildLab' in temp.columns:
        temp.drop(columns='OsBuildLab', inplace=True)

    # Convert object columns to lowercase
    cols = temp.select_dtypes(include=[object]).columns.tolist()
    temp[cols] = temp[cols].apply(lambda x: x.astype(str).str.lower())

    # Simplify SmartScreen values
    smart_screen_cat = 'SmartScreen'
    if smart_screen_cat in temp.columns:
        temp[smart_screen_cat].replace({'promt': 'prompt', '00000000': '0'}, inplace=True)

    # Simplify Census_PrimaryDiskTypeName values
    disk_type_cat = 'Census_PrimaryDiskTypeName'
    if disk_type_cat in temp.columns:
        disk_types = ['HDD', 'SSD']
        temp[disk_type_cat] = temp[disk_type_cat].where(temp[disk_type_cat].isin(disk_types), 'na')

    # Simplify Census_PowerPlatformRoleName values
    role_name_cat = 'Census_PowerPlatformRoleName'
    if role_name_cat in temp.columns:
        na_types = ['unspecified', 'unknown', np.nan]
        temp[role_name_cat] = temp[role_name_cat].where(~temp[role_name_cat].isin(na_types), 'na')

    # Convert categorical columns to 'category' data type
    temp[cols] = temp[cols].astype('category')

    return temp


In [19]:
X = preprocess_cat_values(X)

In [20]:
X.shape, y.shape

((15000, 57), (15000,))

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 57 columns):
 #   Column                                  Non-Null Count  Dtype   
---  ------                                  --------------  -----   
 0   EngineVersion                           15000 non-null  category
 1   AppVersion                              15000 non-null  category
 2   AvSigVersion                            15000 non-null  category
 3   RtpStateBitfield                        14943 non-null  Int16   
 4   IsSxsPassiveMode                        15000 non-null  Int8    
 5   AVProductStatesIdentifier               14941 non-null  float64 
 6   AVProductsInstalled                     14941 non-null  float64 
 7   AVProductsEnabled                       14941 non-null  float64 
 8   HasTpm                                  15000 non-null  Int8    
 9   CountryIdentifier                       15000 non-null  Int64   
 10  CityIdentifier                          14422 

In [22]:
X.isnull().sum()

EngineVersion                                0
AppVersion                                   0
AvSigVersion                                 0
RtpStateBitfield                            57
IsSxsPassiveMode                             0
AVProductStatesIdentifier                   59
AVProductsInstalled                         59
AVProductsEnabled                           59
HasTpm                                       0
CountryIdentifier                            0
CityIdentifier                             578
OrganizationIdentifier                    4548
GeoNameIdentifier                            0
LocaleEnglishNameIdentifier                  0
Platform                                     0
Processor                                    0
OsVer                                        0
OsBuild                                      0
OsSuite                                      0
OsPlatformSubRelease                         0
SkuEdition                                   0
IsProtected  

In [23]:
str_cols = X.select_dtypes(include = 'object').columns
non_str_cols = X.select_dtypes(exclude = 'object').columns

In [24]:
na_value = -1

X[str_cols] = X[str_cols].fillna('na')
X[non_str_cols] = X[non_str_cols].fillna(na_value)
X.isna().sum()

EngineVersion                             0
AppVersion                                0
AvSigVersion                              0
RtpStateBitfield                          0
IsSxsPassiveMode                          0
AVProductStatesIdentifier                 0
AVProductsInstalled                       0
AVProductsEnabled                         0
HasTpm                                    0
CountryIdentifier                         0
CityIdentifier                            0
OrganizationIdentifier                    0
GeoNameIdentifier                         0
LocaleEnglishNameIdentifier               0
Platform                                  0
Processor                                 0
OsVer                                     0
OsBuild                                   0
OsSuite                                   0
OsPlatformSubRelease                      0
SkuEdition                                0
IsProtected                               0
IeVerIdentifier                 

In [25]:
# Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [26]:
# Performing Ordinal Encoding:

unknown_value = -100
oe = OrdinalEncoder(handle_unknown = 'use_encoded_value', dtype = 'int32', unknown_value = unknown_value)
X_train_encoded = oe.fit_transform(X_train)
X_test_encoded = oe.transform(X_test)

max_vals = X_train_encoded.max(axis = 0) + 1
X_test_encoded = np.where(X_test_encoded == unknown_value, max_vals, X_test_encoded)

In [27]:
X_train_encoded.shape, X_test_encoded.shape, y_train.shape, y_test.shape

((12000, 57), (3000, 57), (12000,), (3000,))

In [28]:
# Convert the training and testing data into LightGBM's Dataset format
train_data = lgb.Dataset(X_train_encoded, label=y_train)
test_data = lgb.Dataset(X_test_encoded, label=y_test)

# Define the parameters for Light GBM
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'force_row_wise':True
}

# Train the Light GBM model
model = lgb.train(params, train_data, num_boost_round=100)

# Make predictions on the testing data
y_pred = model.predict(X_test_encoded)
y_pred_binary = [1 if pred >= 0.5 else 0 for pred in y_pred]

# Calculate accuracy on the testing data
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy of the Model:", accuracy*100, '%')


Accuracy of the Model: 62.46666666666667 %
